In [8]:
import ee 
import folium
import geehydro
import json
import requests
from datetime import datetime as dt
from IPython.display import Image

ee.Initialize()

maricopa = [[-113.6318726643051,32.3815643902006],
            [-110.8797974689926,32.3815643902006],
            [-110.8797974689926,34.413007528786],
            [-113.6318726643051,34.413007528786],
            [-113.6318726643051,32.3815643902006]] 

maricopa_AOI = ee.Geometry.Polygon(maricopa)
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
landsat_AOI = landsat.filterBounds(maricopa_AOI).filterDate('2020-01-01','2020-03-31').sort('CLOUD_COVER').first()

phoenix = landsat_AOI

parameters = {'min': 0,
              'max': 3000,
              'scale':30,
              'bands': ['B5', 'B6', 'B4'],
              'gamma': 1.4,
              'crs': 'EPSG:3857',
              'maxPixels': 10e13,
              'region': maricopa_AOI}

parameters2 = {'min': 0,
              'max': 3000,
              'scale':30,
              'bands': ['B4', 'B3', 'B2'],
              'gamma': 1.4,
              'crs': 'EPSG:3857',
              'maxPixels': 10e13,
              'region': maricopa_AOI}

ndvi = phoenix.normalizedDifference(['B5', 'B4'])
palette_ndvi = ['blue', 'white', 'green']
ndvi_parameters = {'min': -1,
                   'max': 1,
                   'scale':30,
                   'palette': palette_ndvi,
                   'crs': 'EPSG:3857',
                   'region': maricopa_AOI}


thermal = phoenix.select('B11').multiply(0.1)

palette_thermal = ['blue','white', 'green']
thermal_parameters = {'min': 291.918,
                      'max': 302.382,
                      'palette': palette_thermal,
                      'crs': 'EPSG:3857',
                      'region': maricopa_AOI}


min = ee.Number(ndvi.reduceRegion(reducer = ee.Reducer.min(), 
                        geometry = maricopa_AOI,
                        scale = 30,
                        maxPixels = 1e9)).getInfo()['nd']

max = ee.Number(ndvi.reduceRegion(reducer = ee.Reducer.max(), 
                        geometry = maricopa_AOI,
                        scale = 30,
                        maxPixels = 1e9)).getInfo()['nd']



fractional_veg = (ndvi.subtract(min).divide(max-min)).pow(ee.Number(2))

a = 0.004
b = 0.986

emissivity = fractional_veg.multiply(a).add(b).rename("EMM")
print(emissivity.getInfo())

LST = thermal.expression(
    '(Tb/(1+(0.00115*(Tb/1.438))*log(Ep)))-273.15', {
        'Tb': thermal.select('B11'),
        'Ep': emissivity.select('EMM'),
    })

LST_palette = ['040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
               '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
               '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
               'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
               'ff0000', 'de0101', 'c21301', 'a71001', '911003']

LST_Params = {'min': 0,
              'max': 50,
              'scale':30,
              'palette': LST_palette,
              'crs': 'EPSG:3857',
              'region': maricopa_AOI}


phoenix_map = folium.Map(location=[33.4484, -112.0740], zoom_start=7)
phoenix_map.addLayer(phoenix, parameters2, "BaseMap")
phoenix_map.addLayer(ndvi, ndvi_parameters, "NDVI")
phoenix_map.addLayer(thermal,thermal_parameters, "Thermal")
phoenix_map.addLayer(LST, LST_Params, name = "LST")
folium.LayerControl().add_to(phoenix_map)

phoenix_map



{'type': 'Image', 'bands': [{'id': 'EMM', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [7751, 7881], 'crs': 'EPSG:32612', 'crs_transform': [30, 0, 242685, 0, -30, 3789615]}]}
